In [2]:
import re
import csv
import json
import pandas as pd
import numpy as np
from collections import defaultdict

# 번역 수정 자동화

In [2]:
# OLD
csvFilePath = r'C:\Users\dh\Desktop\상진\전체재검수\221118_BTC-500_ver.3.0.0.csv'

# NEW
new_csvFilePath = r'C:\Users\dh\Desktop\상진\전체재검수\221118_BTC-500_ver.3.0.1.csv'

In [3]:
df = pd.read_csv(csvFilePath)

with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        skipped_error = []
        
        idx = 0
        for rows in csvReader:
            orig = list(rows.values())[6]
            trans = list(rows.values())[7]
            if 'No evidence' in orig and '증거 없음' not in trans:
                
                name = list(rows.values())[4]

                print("==================================")
                print("csv row #", idx+2)
                print(name)
                print(orig)
                print(trans)
                print("추천: ", trans.replace("없음","증거 없음"))
                choice = input(f'(1)추천/(2)변경무/(3)직접입력: ')

                if choice == "1":
                    print("*** Overwriting as suggested")
                    df.loc[idx,['해석']] = trans.replace("없음","증거 없음")
                elif choice == "2":
                    print("*** No change")
                    continue
                else:
                    edit = input(f'직접입력: ')
                    if edit == "":
                        print("*** SKIPPED ROW #", idx+2)
                        skipped_error.append(name)
                        idx += 1
                        continue
                    df.loc[idx,['해석']] = edit
                print("==================================\n\n")
                df.to_csv(new_csvFilePath, encoding='utf-8', index=False)
            idx += 1
            
df.to_csv(new_csvFilePath, encoding='utf-8', index=False)


csv row # 12992
2330_img-2.jpg-8
No evidence of local lesions at liver, pancreas, both adrenal glands, both kidneys.
간, 췌장, 양측 부신, 양측 신장에 국소 병변 없음.
추천:  간, 췌장, 양측 부신, 양측 신장에 국소 병변 증거 없음.


KeyboardInterrupt: Interrupted by user

# Miscellaneous Data Preprocessing
## Combining 3 datasets and matching their columns, etc.

In [ ]:
csv1=r"C:\Users\dh\Desktop\상진\번역자 리뷰\221125_BTC_1건_이우빈님.csv"
csv2=r"C:\Users\dh\Desktop\상진\번역자 리뷰\221125_BSC_1건_김경빈님.csv"
csv3=r"C:\Users\dh\Desktop\상진\번역자 리뷰\221125_OT_9건_이보람님.csv"
df1 = pd.read_csv(csv1)
df2 = pd.read_csv(csv2)
df3 = pd.read_csv(csv3)
df1 = df1.iloc[:,1:]
df2 = df2.iloc[:,1:]
df3 = df3.iloc[:,1:]
df3 = df3[['result_id', 'file_name', '이미지', '원문', '해석', '카테고리', '약어1','약어2', '약어3', '약어4', '약어5', '약어6', '약어7', '약어8', '약어9', '약어10']]
df3["링크"] = ""
df3 = df3[['result_id', 'file_name', '이미지', '링크', '원문', '해석', '카테고리', '약어1', '약어2', '약어3', '약어4', '약어5', '약어6', '약어7', '약어8', '약어9', '약어10']]
df3 = df3.iloc[:,:-5]
df2 = df2.iloc[:,:-11]
df3.columns = ['샘플폴더','이미지','링크','번호','원문','해석','카테고리','매칭1','매칭2','매칭3','매칭4','매칭5']
df3['샘플폴더'] = df3['샘플폴더'].astype("Int32").astype(str)
df = pd.concat([df1,df2,df3])
df = df.reset_index(drop=True)
df["카테고리"] = ""
df["name"] = ""
df["status"] = ""
df["체크용"] = 0
# df.to_csv(r"C:\Users\dh\Desktop\상진\번역자 리뷰\221125_BTC-600.csv", index=False)
df = df[['샘플폴더', '이미지', '링크', '번호', 'name', 'status', '원문', '해석', '체크용', '카테고리', '매칭1', '매칭2', '매칭3', '매칭4', '매칭5']]
df.loc[df['카테고리']=="", '카테고리'] = '해당없음'
df.loc[df['status']=="", 'status'] = 'BTC-600'
df.loc[pd.isna(df['링크'])==True, '링크'] = '링크'
df.loc[df['링크']=="이미지 바로가기", '링크'] = '링크'
df['번호'] = df['번호'].fillna(0)
df.loc[df['번호']=="", '번호'] = 0
df['번호'] = df['번호'].astype("Int32").astype(str)
df

In [ ]:
df1 = pd.read_csv(r"C:\Users\dh\Desktop\상진\번역자 리뷰\221125_BTC-600.ver.1.0.1.csv")
df2 = pd.read_csv(r"C:\Users\dh\Desktop\상진\번역자 리뷰\221128_10건_이수빈님_검토완료.csv")
df3 = pd.concat([df1, df2])
df3 = df3.reset_index(drop=True)
df3.to_csv(r"C:\Users\dh\Desktop\상진\번역자 리뷰\221128_BTC-600.ver.1.0.0.csv", index=False)

# Checking strings to match syntax ("  "/ ./))

In [36]:
csvFilePath = r"C:\Users\dh\Desktop\상진\BTC-600.csv"

df = pd.read_csv(csvFilePath)

with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        idx = 0
        for row in csvReader:
#             org = row["원문"].split(" ")
#             trans = row["해석"].split(" ")
            org = re.findall(r'\W+', row["원문"])
            trans = re.findall(r'\W+', row["해석"])
#             print(row["샘플폴더"]+"-"+row["이미지"]+"-"+row["번호"])
#             print(trans)
            if not trans or not org: continue
            if org[-1] != trans[-1]:
                print("org: ", org)
                print("trans: ", trans)
                print("==================================")
                print(row["샘플폴더"]+"-"+row["이미지"]+"-"+row["번호"])
                print("원문: '{}'".format(row["원문"]))
                print("해석: '{}'".format(row["해석"]))
#                 edit = " ".join(org[:-1])
#                 print("edit: '{}'".format(edit))
#                 print(len(row["원문"].split(" ")), " vs ",len(row["해석"].split(" ")))
                print("==================================\n")
                df.loc[idx,['원문']] = edit
            idx += 1

df["번호"] = df["번호"].astype("Int32").astype(str)
df["name"] = df["샘플폴더"]+"-"+df["이미지"]+"-"+df["번호"]
df["status"] = "BTC-600"
df = df[['샘플폴더', '이미지', '링크', '번호', 'name', 'status', '원문', '해석', '체크용', '카테고리', '매칭1', '매칭2', '매칭3', '매칭4', 
         '매칭5', '매칭6', '매칭7', '매칭8', '매칭9', '매칭10', '매칭11', '매칭12', '매칭13', '매칭14', '매칭15', '매칭16']]
df            
            

org:  [') ', ': ', '. ']
trans:  [') ', ': ', ', ']
breast_0601522_img-01.jpg-25
원문: '3) Pattern: Solid. Cribriform'
해석: '3) 형태: 고형, 체판'

org:  ['(', ' ) : ', '. ', '% ', ' ', ' ', ', ', ' ', ' ', ' (', ') ', ' ', ' ']
trans:  ['(', ' ) : ', ' ', ' ', ' ', '. ', '% ', ' ', ', ', ' ', ' ', ' (', ')']
breast_0601522_img-01.jpg-31
원문: '(A00 ) : positive. 95% with strong intensity, Allred total score (8) in invasive carcinoma'
해석: '(A00 ) : 침윤성 암종 내 양성. 95% 고강도 동반, Allred 전체 점수 (8)'

org:  ['(', ') : ', '. ', '% ', ' ', ' ', ', ', ' ', ' ', ' (', ') ', ' ', ' ', ' ']
trans:  ['(', ') : ', ' ', ' ', ' ', '. ', '% ', ' ', ', ', ' ', ' ', ' (', ')']
breast_0601522_img-01.jpg-33
원문: '(A00) : positive. 11% with strong intensity, Allred total score (6) in invasi ve carcinoma'
해석: '(A00) : 침윤성 암종 내 양성. 11% 고강도 동반, Allred 전체 점수 (6)'

org:  ['-', ' [', '] ( ', ' ) : ', '+ ', ' ', ' ']
trans:  [' ', ' [', '] (', ') : ', ' ', ' ', ' ', '+']
breast_0601522_img-01.jpg-35
원문: 'C-erbB2 [Breast] ( A00 ) :

org:  ['.', ' ', ': ', ' ', ' ( ', ').']
trans:  ['. ', ' ', ': ', ' ', ' (', ')']
5203_img-4.jpg-19
원문: '2.Slide key: 1 cassette ( representative).'
해석: '2. 슬라이드 키: 카세트 1개 (표본)'

org:  ['.', ': ', ' "', '-', '-', ' ', '"']
trans:  ['.', ': "', '"', ' ']
5203_img-4.jpg-21
원문: '1.Specimen: Labeled "cul-de-sac mass"'
해석: '1.검체: "더슬라스와"로 표시됨'

org:  ['. ', ' ', ': ', ' ', ' (', ').']
trans:  ['. ', ' ', ': ', ' ', ' (', ')']
5203_img-4.jpg-23
원문: '2. Slide keys: 2 cassettes (entire).'
해석: '2. 슬라이드 키: 카세트 2개 (전체)'

org:  ['.', ': ', ' "', '-', '-', '"']
trans:  ['.', ': "', '"', ' ']
5203_img-4.jpg-25
원문: '1.Specimen: Labeled "cul-de-sac"'
해석: '1.검체: "맹낭"으로 표시됨'



,샘플폴더,이미지,링크,번호,name,status,원문,해석,체크용,카테고리,...,매칭7,매칭8,매칭9,매칭10,매칭11,매칭12,매칭13,매칭14,매칭15,매칭16
0,2710_img,1.jpg,링크,1,2710_img-1.jpg-1,BTC-600,"이 문서는 서울아산병원의 문서입니다. 동의없이 수정, 변경 및 복사할 수 없습니다.",NaN,0.0,해당없음,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2710_img,1.jpg,링크,2,2710_img-1.jpg-2,BTC-600,영상의학과 검사 보고서,NaN,0.0,해당없음,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2710_img,1.jpg,링크,3,2710_img-1.jpg-3,BTC-600,병원등록번호 :,NaN,0.0,해당없음,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2710_img,1.jpg,링크,4,2710_img-1.jpg-4,BTC-600,환자성명 :,NaN,0.0,해당없음,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2710_img,1.jpg,링크,5,2710_img-1.jpg-5,BTC-600,생년월일 나이/성별:,NaN,0.0,해당없음,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3298,5203_img,4.jpg,링크,28,5203_img-4.jpg-28,BTC-600,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3299,5203_img,4.jpg,링크,29,5203_img-4.jpg-29,BTC-600,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3300,5203_img,4.jpg,링크,30,5203_img-4.jpg-30,BTC-600,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3301,5203_img,4.jpg,링크,31,5203_img-4.jpg-31,BTC-600,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Excel to CSV (with preprocessing)

In [3]:
excelFilePath = r"C:\Users\dh\Desktop\상진\번역자 리뷰\221206_BTC_100건_아주대6차.xlsx"
df = pd.read_excel(excelFilePath, sheet_name="판독지 해석")
csvFilePath = r"C:\Users\dh\Desktop\상진\번역자 리뷰\221206_BTC_100건_아주대6차.csv"
df.to_csv(r"C:\Users\dh\Desktop\상진\번역자 리뷰\221206_BTC_100건_아주대6차.csv", index=False)
df

C:\Users\dh\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,샘플폴더,이미지,링크,번호,원문,해석,카테고리,매칭1,매칭2,매칭3,...,매칭7,매칭8,매칭9,매칭10,매칭11,매칭12,매칭13,매칭14,매칭15,매칭16
0,1028_img,1.jpg,링크,1,등록번호: 환자명: 성별/나이: (남/72세),등록번호: 환자명: 성별/나이: (남/72세),검사정보,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1028_img,1.jpg,링크,2,수진과: 소화기내과,수진과: 소화기내과,검사정보,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1028_img,1.jpg,링크,3,소견,소견,소견,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1028_img,1.jpg,링크,4,low abd pain 15DA GI Sx:-,하복부통증 15일 전 위장관 증상:-,소견,abd=abdominal,DA=days ago,GI=gastrointestinal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1028_img,1.jpg,링크,5,2021.12.10 PET,2021.12.10 PET,소견,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5132,970_img,2.jpg,링크,8,DIAGNOSIS:,진단:,결론,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5133,970_img,2.jpg,링크,9,"Colon, AV 70 cm, colonoscopic biopsy (No.1);","결장, 항문피부선 70 cm, 결장경하 생검 (1번);",결론,AV=anal verge,No.=number,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5134,970_img,2.jpg,링크,10,Tubular adenoma with low grade dysplasia,저등급 이형성증을 동반한 관상 선종,결론,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5135,970_img,2.jpg,링크,11,"Colon, AV 2 cm, colonoscopic biopsy (No.2);","결장, 항문피부선 2 cm, 결장경하 생검 (2번);",결론,AV=anal verge,No.=number,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df = pd.read_csv(csvFilePath)
with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        idx = 1
        name_prev = ""
        for i,rows in enumerate(csvReader):
            r = list(rows.values())
            folder_curr = r[0]
            img_curr = r[1]
            name_curr = folder_curr+"."+img_curr
            if name_curr != name_prev:
                idx = 1
            name_prev = name_curr
            df.at[i, "번호"] = idx
            idx += 1

df["번호"] = df["번호"].astype("Int32").astype(str)
df["name"] = df["샘플폴더"]+"-"+df["이미지"]+"-"+df["번호"]
df["status"] = ""
df["translator"] = "아주대_221206"
df["체크용"] = 0
df = df[['샘플폴더', '이미지', '링크', '번호', 'name', 'status', 'translator', '원문', '해석', '체크용', '카테고리', '매칭1', '매칭2', '매칭3', '매칭4',
         '매칭5', '매칭6', '매칭7', '매칭8', '매칭9', '매칭10', '매칭11', '매칭12', '매칭13', '매칭14', '매칭15', '매칭16']]

df.to_csv(csvFilePath, encoding='utf-8', index=False)
df

,샘플폴더,이미지,링크,번호,name,status,translator,원문,해석,체크용,...,매칭7,매칭8,매칭9,매칭10,매칭11,매칭12,매칭13,매칭14,매칭15,매칭16
0,1028_img,1.jpg,링크,1,1028_img-1.jpg-1,,아주대_221206,등록번호: 환자명: 성별/나이: (남/72세),등록번호: 환자명: 성별/나이: (남/72세),0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1028_img,1.jpg,링크,2,1028_img-1.jpg-2,,아주대_221206,수진과: 소화기내과,수진과: 소화기내과,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1028_img,1.jpg,링크,3,1028_img-1.jpg-3,,아주대_221206,소견,소견,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1028_img,1.jpg,링크,4,1028_img-1.jpg-4,,아주대_221206,low abd pain 15DA GI Sx:-,하복부통증 15일 전 위장관 증상:-,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1028_img,1.jpg,링크,5,1028_img-1.jpg-5,,아주대_221206,2021.12.10 PET,2021.12.10 PET,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5132,970_img,2.jpg,링크,8,970_img-2.jpg-8,,아주대_221206,DIAGNOSIS:,진단:,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5133,970_img,2.jpg,링크,9,970_img-2.jpg-9,,아주대_221206,"Colon, AV 70 cm, colonoscopic biopsy (No.1);","결장, 항문피부선 70 cm, 결장경하 생검 (1번);",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5134,970_img,2.jpg,링크,10,970_img-2.jpg-10,,아주대_221206,Tubular adenoma with low grade dysplasia,저등급 이형성증을 동반한 관상 선종,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5135,970_img,2.jpg,링크,11,970_img-2.jpg-11,,아주대_221206,"Colon, AV 2 cm, colonoscopic biopsy (No.2);","결장, 항문피부선 2 cm, 결장경하 생검 (2번);",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 약어별 풀이 JSON 생성
## Sort acronyms

In [11]:
csvFilePath = r"C:\Users\dh\Desktop\상진\BTC.csv"
jsonFilePath = r"C:\Users\dh\Desktop\상진\acronyms.json"

df = pd.read_csv(csvFilePath)
ac = defaultdict(list)
with open(csvFilePath, encoding='utf-8') as csvf:
    csvReader = csv.DictReader(csvf)
    for row in csvReader:
        filter_cols = ['매칭1', '매칭2', '매칭3', '매칭4', '매칭5', 
                        '매칭6', '매칭7', '매칭8', '매칭9', '매칭10', 
                        '매칭11', '매칭12', '매칭13', '매칭14', '매칭15', 
                        '매칭16']
        new_rows = { col: row[col] for col in filter_cols }
        present_cols = [v for k, v in new_rows.items() if v != '']
        for i in present_cols:
#             print(row['name'])
            acro, defi = i.split("=")[0], i.split("=")[1]
            if defi == "gemcitabine and nab\u2010paclitaxel":
                print(row)
            if defi not in ac[acro]:
                ac[acro].append(defi)
                
ac = dict(ac)
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(ac, indent=4))
        

C:\Users\dh\AppData\Local\Temp\ipykernel_7692\4063204230.py:4: DtypeWarning: Columns (5,9,16,17,18,19,20,21,22,23,24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csvFilePath)


# Combined BTC (500 + 600)

In [58]:
new = r"C:\Users\dh\Desktop\상진\BTC.csv"

btc5 = r"C:\Users\dh\Desktop\상진\BTC-500.csv"
btc6 = r"C:\Users\dh\Desktop\상진\BTC-600.csv"
df5 = pd.read_csv(btc5)
df6 = pd.read_csv(btc6)

df = pd.concat([df5, df6], ignore_index=True, sort=False)

df["translator"] = ''
df = df[['샘플폴더', '이미지', '링크', '번호', 'name', 'status', 'translator','원문', '해석', '체크용', '카테고리', '매칭1', '매칭2', '매칭3', '매칭4',
         '매칭5', '매칭6', '매칭7', '매칭8', '매칭9', '매칭10', '매칭11', '매칭12', '매칭13', '매칭14', '매칭15', '매칭16']]

df.loc[df['translator']=="train_002", 'status'] = 'train_003'

df.loc[df['status']=="train_002", 'translator'] = '아주대'
df.loc[df['status']=="train_001", 'translator'] = '아주대'
df.loc[df['status']=="train_003", 'translator'] = '아주대'
df.loc[df['status']=="train_005", 'translator'] = '아주대'
df.loc[df['status']=="test_005", 'translator'] = '아주대'
df.loc[df['status']=="test_003", 'translator'] = '아주대'
df.loc[df['status']=="test_002", 'translator'] = '아주대'
df.loc[df['status']=="test_001", 'translator'] = '아주대'

df.loc[df['status']=="BTC-600", 'translator'] = 'ontol'

df.loc[df['status']=="train_002", 'status'] = 'train_003'
df.loc[df['status']=="train_001", 'status'] = 'train_003'
df.loc[df['status']=="test_002", 'status'] = 'test_003'
df.loc[df['status']=="test_001", 'status'] = 'test_003'
df.loc[df['status']=="BTC-600", 'status'] = ''

df
df.to_csv(new, encoding='utf-8', index=False)


# Ranking System for the FIFA tournament

In [7]:
# create a list of players
players = ["player1", "player2", "player3", "player4", "player5", "player6", "player7", "player8", "player9", "player10"]

# create a dictionary to store scores for each player
scores = {}
for player in players:
    scores[player] = {"individual_scores": [], "team_scores": []}

# function to calculate ranking based on scores
def calculate_ranking(scores):
    rankings = {}
    for player, score in scores.items():
        total_score = sum(score["individual_scores"]) + sum(score["team_scores"]) * 2 # give more weight to team scores
        rankings[player] = total_score

    # sort rankings in descending order
    sorted_rankings = sorted(rankings.items(), key=lambda x: x[1], reverse=True)
    return sorted_rankings

# function to update scores based on match results
def update_scores(player1, player2, player1_score, player2_score):
    # update individual scores
    scores[player1]["individual_scores"].append(player1_score)
    scores[player2]["individual_scores"].append(player2_score)

# function to update scores for team matches
def update_team_scores(team1, team2, team1_score, team2_score):
    # update team scores
    for player in team1:
        scores[player]["team_scores"].append(team1_score)
    for player in team2:
        scores[player]["team_scores"].append(team2_score)

# function to display scoreboard
def display_scoreboard():
    rankings = calculate_ranking(scores)
    print("Rankings:")
    for i, (player, score) in enumerate(rankings):
        print(f"{i+1}. {player}: {score}")

# example match results
update_scores("player1", "player2", 3, 1)
update_scores("player3", "player4", 2, 2)
update_scores("player5", "player6", 1, 3)
update_scores("player7", "player8", 0, 4)
update_scores("player9", "player10", 3, 3)
update_team_scores(["player1", "player2"], ["player3", "player4"], 2, 0)
update_team_scores(["player5", "player6"], ["player7", "player8"], 4, 1)

# display scoreboard
display_scoreboard()

# output:
# Rankings:
# 1. player1: 9
# 2. player2: 5
# 3. player6: 5
# 4. player5: 5
# 5. player3: 4
# 6. player4: 2
# 7. player9: 3
# 8. player10: 3
# 9. player7: 1
# 10. player8: 0

Rankings:
1. player6: 11
2. player5: 9
3. player1: 7
4. player8: 6
5. player2: 5
6. player9: 3
7. player10: 3
8. player3: 2
9. player4: 2
10. player7: 2
